In [2]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")

if not API_KEY:
    raise RuntimeError("Brak GOOGLE_MAPS_API_KEY w .env")

def transit_route(origin: str, destination: str):
    url = "https://maps.googleapis.com/maps/api/directions/json"

    params = {
        "origin": origin,
        "destination": destination,
        "mode": "transit",          # 🔑 komunikacja miejska
        # opcjonalnie możesz ograniczyć typy:
        # "transit_mode": "bus|subway|train|tram|rail",
        "key": API_KEY
    }

    r = requests.get(url, params=params)
    print("Status code:", r.status_code)
    data = r.json()
    print("Raw response status:", data.get("status"))

    if data.get("status") != "OK":
        raise RuntimeError(data.get("error_message", "Directions failed"))

    route = data["routes"][0]
    leg = route["legs"][0]

    print("\nOdległość:", leg["distance"]["text"])
    print("Czas:", leg["duration"]["text"])

    print("\nKroki trasy (środki transportu):")
    for i, step in enumerate(leg["steps"], start=1):
        travel_mode = step["travel_mode"]
        if travel_mode == "WALKING":
            print(f"{i}. PIESZO – {step['distance']['text']}")
        elif travel_mode == "TRANSIT":
            t = step["transit_details"]
            line = t["line"]
            vehicle_type = line["vehicle"]["type"]  # BUS / TRAM / SUBWAY / TRAIN / RAIL
            line_name = line.get("short_name") or line.get("name")
            num_stops = t["num_stops"]
            dep_stop = t["departure_stop"]["name"]
            arr_stop = t["arrival_stop"]["name"]
            print(
                f"{i}. {vehicle_type}: linia {line_name}, "
                f"z {dep_stop} do {arr_stop} ({num_stops} przystanków)"
            )
        else:
            print(f"{i}. {travel_mode}")

if __name__ == "__main__":
    # podmień na jakieś sensowne lokalne adresy
    transit_route("Łódź, Poland", "Warsaw, Poland")


Status code: 200
Raw response status: OK

Odległość: 137 km
Czas: 1 hour 28 mins

Kroki trasy (środki transportu):
1. HEAVY_RAIL: linia LKA, z Łódź Kaliska do Łódź Chojny (2 przystanków)
2. HEAVY_RAIL: linia IC6126, z Łódź Chojny do Warsaw West (5 przystanków)


In [3]:
transit_route("Al. Politechniki 3, Łódź", "Ogrodowa 15, Łódź")

Status code: 200
Raw response status: OK

Odległość: 3.8 km
Czas: 21 mins

Kroki trasy (środki transportu):
1. PIESZO – 0.2 km
2. TRAM: linia 15, z Politechniki - Targi Łódzkie (2123) do Legionów - Zachodnia (0637) (8 przystanków)
3. PIESZO – 0.3 km
